In [24]:
import cv2
import numpy as np
from pyzbar.pyzbar import decode
import os


In [12]:
def preprocess_image(image, method="sharpen"):
    if method == "blur":
        return cv2.GaussianBlur(image, (5, 5), 0)
    elif method == "sharpen":
        kernel = np.array([[0, -1, 0],
                           [-1, 5, -1],
                           [0, -1, 0]])
        return cv2.filter2D(image, -1, kernel)
    return image

In [13]:
def decode_and_draw(image):
    decoded_objects = decode(image)
    for obj in decoded_objects:
        try:
            barcode_data = obj.data.decode("utf-8")
        except Exception as e:
            barcode_data = "<undecodable>"
            print(f"Decode error: {e}")

        barcode_type = obj.type if obj.type else "<unknown>"

        print("Type:", barcode_type)
        print("Data:", barcode_data)
        print("Bounding Box:", obj.rect)

        points = obj.polygon
        if len(points) > 4:
            pts = np.array([(pt.x, pt.y) for pt in points], dtype=np.int32)
            pts = pts.reshape((-1, 1, 2))
            hull = cv2.convexHull(pts).squeeze()
        else:
            hull = [(pt.x, pt.y) for pt in points]

        for j in range(len(hull)):
            pt1 = tuple(hull[j])
            pt2 = tuple(hull[(j + 1) % len(hull)])
            cv2.line(image, pt1, pt2, (255, 0, 0), 3)
    return image

In [14]:
def run_pipeline(image_path, preprocess="sharpen", show=True):
    if not os.path.exists(image_path):
        print("Image not found.")
        return

    original = cv2.imread(image_path)
    processed = original.copy()

    processed = preprocess_image(processed, method=preprocess)
    processed = decode_and_draw(processed)

    if show:
        height = 500
        original_resized = cv2.resize(original, (int(original.shape[1] * height / original.shape[0]), height))
        processed_resized = cv2.resize(processed, (int(processed.shape[1] * height / processed.shape[0]), height))

        combined = np.hstack((original_resized, processed_resized))

        cv2.imshow("Input (Left) vs Output (Right)", combined)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

Testing

In [20]:
test_images = [r"C:\Users\shail\OneDrive\Documents\Study material\AIP\Barcode input\barcode 1.webp"]

run_pipeline(test_images[0], preprocess="sharpen")

Type: EAN13
Data: 9336672318343
Bounding Box: Rect(left=142, top=150, width=158, height=97)


In [22]:
test_images = [r"C:\Users\shail\OneDrive\Documents\Study material\AIP\Barcode input\barcode 2b.png"]

run_pipeline(test_images[0], preprocess="sharpen")

Type: EAN13
Data: 0705632441947
Bounding Box: Rect(left=24, top=0, width=206, height=122)


In [25]:
test_images = [r"C:\Users\shail\OneDrive\Documents\Study material\AIP\Barcode input\barcode 3b.png"]

run_pipeline(test_images[0], preprocess="sharpen")

Type: EAN13
Data: 0860003773741
Bounding Box: Rect(left=336, top=109, width=638, height=452)
